**Data cleaning tasks for [Global Terrorism Dataset](https://www.kaggle.com/START-UMD/gtd)**


---



---




In [1]:
import pandas as pd
import numpy as np

from google.colab import drive
from google.colab import files

### Reading data file from my personal Google Drive

In [2]:
drive.mount('/content/drive')
GTD_data = pd.read_csv("/content/drive/MyDrive/data_warehouse_project/GTD_data.csv", encoding ='ISO-8859-1', low_memory=False)

Mounted at /content/drive


### General info about the data file and presentation of file's content

In [3]:
GTD_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181691 entries, 0 to 181690
Columns: 135 entries, eventid to related
dtypes: float64(55), int64(22), object(58)
memory usage: 187.1+ MB


In [4]:
GTD_data

,eventid,iyear,imonth,iday,approxdate,extended,resolution,country,country_txt,region,region_txt,provstate,city,latitude,longitude,specificity,vicinity,location,summary,crit1,crit2,crit3,doubtterr,alternative,alternative_txt,multiple,success,suicide,attacktype1,attacktype1_txt,attacktype2,attacktype2_txt,attacktype3,attacktype3_txt,targtype1,targtype1_txt,targsubtype1,targsubtype1_txt,corp1,target1,...,weapsubtype4,weapsubtype4_txt,weapdetail,nkill,nkillus,nkillter,nwound,nwoundus,nwoundte,property,propextent,propextent_txt,propvalue,propcomment,ishostkid,nhostkid,nhostkidus,nhours,ndays,divert,kidhijcountry,ransom,ransomamt,ransomamtus,ransompaid,ransompaidus,ransomnote,hostkidoutcome,hostkidoutcome_txt,nreleased,addnotes,scite1,scite2,scite3,dbsource,INT_LOG,INT_IDEO,INT_MISC,INT_ANY,related
0,197000000001,1970,7,2,NaN,0,NaN,58,Dominican Republic,2,Central America & Caribbean,NaN,Santo Domingo,18.456792,-69.951164,1.0,0,NaN,NaN,1,1,1,0.0,NaN,NaN,0.0,1,0,1,Assassination,NaN,NaN,NaN,NaN,14,Private Citizens & Property,68.0,Named Civilian,NaN,Julio Guzman,...,NaN,NaN,NaN,1.0,NaN,NaN,0.0,NaN,NaN,0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PGIS,0,0,0,0,NaN
1,197000000002,1970,0,0,NaN,0,NaN,130,Mexico,1,North America,Federal,Mexico city,19.371887,-99.086624,1.0,0,NaN,NaN,1,1,1,0.0,NaN,NaN,0.0,1,0,6,Hostage Taking (Kidnapping),NaN,NaN,NaN,NaN,7,Government (Diplomatic),45.0,"Diplomatic Personnel (outside of embassy, cons...",Belgian Ambassador Daughter,"Nadine Chaval, daughter",...,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0,NaN,NaN,NaN,NaN,1.0,1.0,0.0,NaN,NaN,NaN,Mexico,1.0,800000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PGIS,0,1,1,1,NaN
2,197001000001,1970,1,0,NaN,0,NaN,160,Philippines,5,Southeast Asia,Tarlac,Unknown,15.478598,120.599741,4.0,0,NaN,NaN,1,1,1,0.0,NaN,NaN,0.0,1,0,1,Assassination,NaN,NaN,NaN,NaN,10,Journalists & Media,54.0,Radio Journalist/Staff/Facility,Voice of America,Employee,...,NaN,NaN,NaN,1.0,NaN,NaN,0.0,NaN,NaN,0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PGIS,-9,-9,1,1,NaN
3,197001000002,1970,1,0,NaN,0,NaN,78,Greece,8,Western Europe,Attica,Athens,37.997490,23.762728,1.0,0,NaN,NaN,1,1,1,0.0,NaN,NaN,0.0,1,0,3,Bombing/Explosion,NaN,NaN,NaN,NaN,7,Government (Diplomatic),46.0,Embassy/Consulate,NaN,U.S. Embassy,...,NaN,NaN,Explosive,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PGIS,-9,-9,1,1,NaN
4,197001000003,1970,1,0,NaN,0,NaN,101,Japan,4,East Asia,Fukouka,Fukouka,33.580412,130.396361,1.0,0,NaN,NaN,1,1,1,-9.0,NaN,NaN,0.0,1,0,7,Facility/Infrastructure Attack,NaN,NaN,NaN,NaN,7,Government (Diplomatic),46.0,Embassy/Consulate,NaN,U.S. Consulate,...,NaN,NaN,Incendiary,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PGIS,-9,-9,1,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181686,201712310022,2017,12,31,NaN,0,NaN,182,Somalia,11,Sub-Saharan Africa,Middle Shebelle,Ceelka Geelow,2.359673,45.385034,2.0,0,The incident occurred near the town of Balcad.,12/31/2017: Assailants opened fire on a Somali...,1,1,0,1.0,1.0,Insurgency/Guerilla Action,0.0,1,0,2,Armed Assault,NaN,NaN,NaN,NaN,4,Military,36.0,Military Checkpoint,Somali National Army (SNA),Checkpoint,...,NaN,NaN,NaN,1.0,0.0,0.0,2.0,0.0,0.0,-9,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""Somalia: Al-Shabaab Militants Attack Army Che...","""Highlights: Somalia Daily Media Highlights 2 ...","""Highlights: Somalia Daily Media Highlights 1 ...",START Primary Collection,0,0,0,0,NaN
181687,201712310029,20

### Making sure that there is no duplicate data based in unique event id

In [5]:
uniqueValues = GTD_data['eventid'].unique()
print(len(uniqueValues) == len(GTD_data))

True


### Manual analysis of the dataset and removing useless and irrelevant columns (analysis based on the official [codebook](https://www.start.umd.edu/gtd/downloads/Codebook.pdf))

In [6]:
rm_cols = ["approxdate", "resolution", "country", "region", "specificity", "summary", "vicinity", "location", "alternative", "attacktype1", "attacktype2", "attacktype3", "targtype1", "targsubtype1", "natlty1", "targtype2", "targsubtype2", "natlty2", "targtype3", "targsubtype3", "corp1", "corp2", "corp3", "natlty3", "gsubname", "gname2", "gsubname2", "gname3", "gsubname3", "motive", "guncertain1", "guncertain2", "guncertain3", "individual", "claimmode", "claim2", "claimmode2", "claimmode2_txt", "claim3", "claimmode3", "claimmode3_txt", "compclaim", "weaptype1", "weapsubtype1", "weapsubtype1_txt", "weaptype2", "weapsubtype2", "weapsubtype2_txt", "weaptype3", "weapsubtype3", "weapsubtype3_txt", "weaptype4", "weapsubtype4", "weapsubtype4_txt", "weapdetail", "propextent", "propvalue", "propcomment", "divert", "ransomamtus", "ransompaidus", "ransomnote","hostkidoutcome", "addnotes", "scite1", "scite2", "scite3", "dbsource", "INT_ANY", "INT_MISC", "related"]

In [7]:
for col in rm_cols:
  GTD_data.pop(col)

In [8]:
GTD_data.info()
GTD_data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181691 entries, 0 to 181690
Data columns (total 64 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   eventid             181691 non-null  int64  
 1   iyear               181691 non-null  int64  
 2   imonth              181691 non-null  int64  
 3   iday                181691 non-null  int64  
 4   extended            181691 non-null  int64  
 5   country_txt         181691 non-null  object 
 6   region_txt          181691 non-null  object 
 7   provstate           181270 non-null  object 
 8   city                181257 non-null  object 
 9   latitude            177135 non-null  float64
 10  longitude           177134 non-null  float64
 11  crit1               181691 non-null  int64  
 12  crit2               181691 non-null  int64  
 13  crit3               181691 non-null  int64  
 14  doubtterr           181690 non-null  float64
 15  alternative_txt     29011 non-null

,eventid,iyear,imonth,iday,extended,country_txt,region_txt,provstate,city,latitude,longitude,crit1,crit2,crit3,doubtterr,alternative_txt,multiple,success,suicide,attacktype1_txt,attacktype2_txt,attacktype3_txt,targtype1_txt,targsubtype1_txt,target1,natlty1_txt,targtype2_txt,targsubtype2_txt,target2,natlty2_txt,targtype3_txt,targsubtype3_txt,target3,natlty3_txt,gname,nperps,nperpcap,claimed,claimmode_txt,weaptype1_txt,weaptype2_txt,weaptype3_txt,weaptype4_txt,nkill,nkillus,nkillter,nwound,nwoundus,nwoundte,property,propextent_txt,ishostkid,nhostkid,nhostkidus,nhours,ndays,kidhijcountry,ransom,ransomamt,ransompaid,hostkidoutcome_txt,nreleased,INT_LOG,INT_IDEO
0,197000000001,1970,7,2,0,Dominican Republic,Central America & Caribbean,NaN,Santo Domingo,18.456792,-69.951164,1,1,1,0.0,NaN,0.0,1,0,Assassination,NaN,NaN,Private Citizens & Property,Named Civilian,Julio Guzman,Dominican Republic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MANO-D,NaN,NaN,NaN,NaN,Unknown,NaN,NaN,NaN,1.0,NaN,NaN,0.0,NaN,NaN,0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0,0
1,197000000002,1970,0,0,0,Mexico,North America,Federal,Mexico city,19.371887,-99.086624,1,1,1,0.0,NaN,0.0,1,0,Hostage Taking (Kidnapping),NaN,NaN,Government (Diplomatic),"Diplomatic Personnel (outside of embassy, cons...","Nadine Chaval, daughter",Belgium,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23rd of September Communist League,7.0,NaN,NaN,NaN,Unknown,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0,NaN,1.0,1.0,0.0,NaN,NaN,Mexico,1.0,800000.0,NaN,NaN,NaN,0,1
2,197001000001,1970,1,0,0,Philippines,Southeast Asia,Tarlac,Unknown,15.478598,120.599741,1,1,1,0.0,NaN,0.0,1,0,Assassination,NaN,NaN,Journalists & Media,Radio Journalist/Staff/Facility,Employee,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown,NaN,NaN,NaN,NaN,Unknown,NaN,NaN,NaN,1.0,NaN,NaN,0.0,NaN,NaN,0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,-9,-9
3,197001000002,1970,1,0,0,Greece,Western Europe,Attica,Athens,37.997490,23.762728,1,1,1,0.0,NaN,0.0,1,0,Bombing/Explosion,NaN,NaN,Government (Diplomatic),Embassy/Consulate,U.S. Embassy,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown,NaN,NaN,NaN,NaN,Explosives,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,-9,-9
4,197001000003,1970,1,0,0,Japan,East Asia,Fukouka,Fukouka,33.580412,130.396361,1,1,1,-9.0,NaN,0.0,1,0,Facility/Infrastructure Attack,NaN,NaN,Government (Diplomatic),Embassy/Consulate,U.S. Consulate,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown,NaN,NaN,NaN,NaN,Incendiary,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,-9,-9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181686,201712310022,2017,12,31,0,Somalia,Sub-Saharan Africa,Middle Shebelle,Ceelka Geelow,2.359673,45.385034,1,1,0,1.0,Insurgency/Guerilla Action,0.0,1,0,Armed Assault,NaN,NaN,Military,Military Checkpoint,Checkpoint,Somalia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Al-Shabaab,-99.0,0.0,1.0,Unknown,Firearms,NaN,NaN,NaN,1.0,0.0,0.0,2.0,0.0,0.0,-9,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
181687,201712310029,2017,12,31,0,Syria,Middle East & North Africa,Lattakia,Jableh,35.407278,35.942679,1,1,0,1.0,Insurgency/Guerilla Action,0.0,1,0,Bombing/Explosion,NaN,NaN,Military,Military Barracks/Base/Headquarters/Checkpost,Hmeymim Air Base,Russia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Muslim extremists,-99.0,0.0,0.0,NaN,Explosives,NaN,NaN,NaN,2.0,0.0,0.0,7.0,0.0,0.0,1,Unknown,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-9,-9
181688,201712310030,2017,12,31,0,Philippines,Southeast Asia,Maguindanao,Kubentog,6.900742,124.437908,1,1,1,0.0,NaN,0.0,1,0,Facility/Infrastructure Attack,NaN,NaN,Private Citizens & Property,House/Apartment/Residence,Houses,Philippines,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bangsamoro Islamic Freedom Movement (BIFM),-99.0,0.0,0.0,NaN,Incendia

### Creation of a function to determine percantage of missing values for each column and removing columns for which calucated value is equal or grater than 80%

In [9]:
def null_vals(df):
    df_null = pd.DataFrame()
    for col in df.columns:
        df_null.loc[col, 'name'] = df[col].name
        df_null.loc[col, 'null values [%]'] = round(df[col].isnull().sum()*100/ len(df),2)
        df_null.loc[col, 'dtype'] = df[col].dtype
    
    return df_null.sort_values(ascending = False, by = 'null values [%]')

In [10]:
df_null = null_vals(GTD_data)
cols_null = []
for i in df_null.index:
  if df_null['null values [%]'][i] >= 80:
    cols_null.append(df_null['name'][i])
print(cols_null)

['weaptype4_txt', 'attacktype3_txt', 'ransompaid', 'targsubtype3_txt', 'natlty3_txt', 'targtype3_txt', 'target3', 'ransomamt', 'weaptype3_txt', 'kidhijcountry', 'nhours', 'attacktype2_txt', 'ndays', 'nreleased', 'targsubtype2_txt', 'natlty2_txt', 'hostkidoutcome_txt', 'target2', 'targtype2_txt', 'weaptype2_txt', 'nhostkidus', 'nhostkid', 'claimmode_txt', 'alternative_txt']


In [11]:
for i in cols_null:
  GTD_data.pop(i)

In [12]:
GTD_data.info()
null_vals(GTD_data)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181691 entries, 0 to 181690
Data columns (total 40 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   eventid           181691 non-null  int64  
 1   iyear             181691 non-null  int64  
 2   imonth            181691 non-null  int64  
 3   iday              181691 non-null  int64  
 4   extended          181691 non-null  int64  
 5   country_txt       181691 non-null  object 
 6   region_txt        181691 non-null  object 
 7   provstate         181270 non-null  object 
 8   city              181257 non-null  object 
 9   latitude          177135 non-null  float64
 10  longitude         177134 non-null  float64
 11  crit1             181691 non-null  int64  
 12  crit2             181691 non-null  int64  
 13  crit3             181691 non-null  int64  
 14  doubtterr         181690 non-null  float64
 15  multiple          181690 non-null  float64
 16  success           18

,name,null values [%],dtype
propextent_txt,propextent_txt,64.74,object
ransom,ransom,57.41,float64
nperps,nperps,39.14,float64
nperpcap,nperpcap,38.25,float64
nwoundte,nwoundte,38.06,float64
nkillter,nkillter,36.85,float64
claimed,claimed,36.39,float64
nwoundus,nwoundus,35.61,float64
nkillus,nkillus,35.47,float64
nwound,nwound,8.98,float64


### Analysis of empty values and converting them into more convinient form to perform Data Warehouse tasks

In [13]:
GTD_data

,eventid,iyear,imonth,iday,extended,country_txt,region_txt,provstate,city,latitude,longitude,crit1,crit2,crit3,doubtterr,multiple,success,suicide,attacktype1_txt,targtype1_txt,targsubtype1_txt,target1,natlty1_txt,gname,nperps,nperpcap,claimed,weaptype1_txt,nkill,nkillus,nkillter,nwound,nwoundus,nwoundte,property,propextent_txt,ishostkid,ransom,INT_LOG,INT_IDEO
0,197000000001,1970,7,2,0,Dominican Republic,Central America & Caribbean,NaN,Santo Domingo,18.456792,-69.951164,1,1,1,0.0,0.0,1,0,Assassination,Private Citizens & Property,Named Civilian,Julio Guzman,Dominican Republic,MANO-D,NaN,NaN,NaN,Unknown,1.0,NaN,NaN,0.0,NaN,NaN,0,NaN,0.0,0.0,0,0
1,197000000002,1970,0,0,0,Mexico,North America,Federal,Mexico city,19.371887,-99.086624,1,1,1,0.0,0.0,1,0,Hostage Taking (Kidnapping),Government (Diplomatic),"Diplomatic Personnel (outside of embassy, cons...","Nadine Chaval, daughter",Belgium,23rd of September Communist League,7.0,NaN,NaN,Unknown,0.0,NaN,NaN,0.0,NaN,NaN,0,NaN,1.0,1.0,0,1
2,197001000001,1970,1,0,0,Philippines,Southeast Asia,Tarlac,Unknown,15.478598,120.599741,1,1,1,0.0,0.0,1,0,Assassination,Journalists & Media,Radio Journalist/Staff/Facility,Employee,United States,Unknown,NaN,NaN,NaN,Unknown,1.0,NaN,NaN,0.0,NaN,NaN,0,NaN,0.0,0.0,-9,-9
3,197001000002,1970,1,0,0,Greece,Western Europe,Attica,Athens,37.997490,23.762728,1,1,1,0.0,0.0,1,0,Bombing/Explosion,Government (Diplomatic),Embassy/Consulate,U.S. Embassy,United States,Unknown,NaN,NaN,NaN,Explosives,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,0.0,0.0,-9,-9
4,197001000003,1970,1,0,0,Japan,East Asia,Fukouka,Fukouka,33.580412,130.396361,1,1,1,-9.0,0.0,1,0,Facility/Infrastructure Attack,Government (Diplomatic),Embassy/Consulate,U.S. Consulate,United States,Unknown,NaN,NaN,NaN,Incendiary,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,0.0,0.0,-9,-9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181686,201712310022,2017,12,31,0,Somalia,Sub-Saharan Africa,Middle Shebelle,Ceelka Geelow,2.359673,45.385034,1,1,0,1.0,0.0,1,0,Armed Assault,Military,Military Checkpoint,Checkpoint,Somalia,Al-Shabaab,-99.0,0.0,1.0,Firearms,1.0,0.0,0.0,2.0,0.0,0.0,-9,NaN,0.0,NaN,0,0
181687,201712310029,2017,12,31,0,Syria,Middle East & North Africa,Lattakia,Jableh,35.407278,35.942679,1,1,0,1.0,0.0,1,0,Bombing/Explosion,Military,Military Barracks/Base/Headquarters/Checkpost,Hmeymim Air Base,Russia,Muslim extremists,-99.0,0.0,0.0,Explosives,2.0,0.0,0.0,7.0,0.0,0.0,1,Unknown,0.0,NaN,-9,-9
181688,201712310030,2017,12,31,0,Philippines,Southeast Asia,Maguindanao,Kubentog,6.900742,124.437908,1,1,1,0.0,0.0,1,0,Facility/Infrastructure Attack,Private Citizens & Property,House/Apartment/Residence,Houses,Philippines,Bangsamoro Islamic Freedom Movement (BIFM),-99.0,0.0,0.0,Incendiary,0.0,0.0,0.0,0.0,0.0,0.0,1,Unknown,0.0,NaN,0,0
181689,201712310031,2017,12,31,0,India,South Asia,Manipur,Imphal,24.798346,93.940430,1,1,1,0.0,0.0,0,0,Bombing/Explosion,Government (General),Government Building/Facility/Office,Office,India,Unknown,-99.0,0.0,0.0,Explosives,0.0,0.0,0.0,0.0,0.0,0.0,-9,NaN,0.0,NaN,-9,-9


In [14]:
df_missing = null_vals(GTD_data)
cols_missing = []
for i in df_missing.index:
  if df_missing['null values [%]'][i] != 0:
    cols_missing.append(df_null['name'][i])
print(cols_missing)

['propextent_txt', 'ransom', 'nperps', 'nperpcap', 'nwoundte', 'nkillter', 'claimed', 'nwoundus', 'nkillus', 'nwound', 'targsubtype1_txt', 'nkill', 'longitude', 'latitude', 'natlty1_txt', 'target1', 'city', 'provstate', 'ishostkid']


In [15]:
for name in cols_missing:
  print(GTD_data[name].dtype)

object
float64
float64
float64
float64
float64
float64
float64
float64
float64
object
float64
float64
float64
object
object
object
object
float64




- ransom - -10 (not applicable)


---

**-99:**
- ishostkid (-9 or -99)
- nperps (-9 or-99)
- nperpcap (-9 or-99)
- nwoundte
- nkillter
- claimed
- nwoundus
- nkillus
- nwound
- nkill
- longitude
- latitude


---

**"Unknown":**
- targsubtype1_txt
- natlty1_txt
- target1
- city
- provstate
- propextent_txt



In [16]:
# removing "ransom" column from the list as it is a special case
cols_missing.remove('ransom')

In [17]:
# converting NaN values in columns based on their data types:
# float64 - NaN -> -99
# string/obj -> "Unknown"

for col in cols_missing:
  if GTD_data[col].dtype == np.float64:
    GTD_data[col] = GTD_data[col].fillna(-99)
  else:
    GTD_data[col] = GTD_data[col].fillna("Unknown")

In [18]:
# manually conver NaN values in "ransom" column
GTD_data['ransom'] = GTD_data['ransom'].fillna(-10)

In [19]:
# check percentage of empty values in columns after convertions 
null_vals(GTD_data)

,name,null values [%],dtype
eventid,eventid,0.0,int64
iyear,iyear,0.0,int64
natlty1_txt,natlty1_txt,0.0,object
gname,gname,0.0,object
nperps,nperps,0.0,float64
nperpcap,nperpcap,0.0,float64
claimed,claimed,0.0,float64
weaptype1_txt,weaptype1_txt,0.0,object
nkill,nkill,0.0,float64
nkillus,nkillus,0.0,float64


In [20]:
GTD_data

,eventid,iyear,imonth,iday,extended,country_txt,region_txt,provstate,city,latitude,longitude,crit1,crit2,crit3,doubtterr,multiple,success,suicide,attacktype1_txt,targtype1_txt,targsubtype1_txt,target1,natlty1_txt,gname,nperps,nperpcap,claimed,weaptype1_txt,nkill,nkillus,nkillter,nwound,nwoundus,nwoundte,property,propextent_txt,ishostkid,ransom,INT_LOG,INT_IDEO
0,197000000001,1970,7,2,0,Dominican Republic,Central America & Caribbean,Unknown,Santo Domingo,18.456792,-69.951164,1,1,1,0.0,0.0,1,0,Assassination,Private Citizens & Property,Named Civilian,Julio Guzman,Dominican Republic,MANO-D,-99.0,-99.0,-99.0,Unknown,1.0,-99.0,-99.0,0.0,-99.0,-99.0,0,Unknown,0.0,0.0,0,0
1,197000000002,1970,0,0,0,Mexico,North America,Federal,Mexico city,19.371887,-99.086624,1,1,1,0.0,0.0,1,0,Hostage Taking (Kidnapping),Government (Diplomatic),"Diplomatic Personnel (outside of embassy, cons...","Nadine Chaval, daughter",Belgium,23rd of September Communist League,7.0,-99.0,-99.0,Unknown,0.0,-99.0,-99.0,0.0,-99.0,-99.0,0,Unknown,1.0,1.0,0,1
2,197001000001,1970,1,0,0,Philippines,Southeast Asia,Tarlac,Unknown,15.478598,120.599741,1,1,1,0.0,0.0,1,0,Assassination,Journalists & Media,Radio Journalist/Staff/Facility,Employee,United States,Unknown,-99.0,-99.0,-99.0,Unknown,1.0,-99.0,-99.0,0.0,-99.0,-99.0,0,Unknown,0.0,0.0,-9,-9
3,197001000002,1970,1,0,0,Greece,Western Europe,Attica,Athens,37.997490,23.762728,1,1,1,0.0,0.0,1,0,Bombing/Explosion,Government (Diplomatic),Embassy/Consulate,U.S. Embassy,United States,Unknown,-99.0,-99.0,-99.0,Explosives,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,1,Unknown,0.0,0.0,-9,-9
4,197001000003,1970,1,0,0,Japan,East Asia,Fukouka,Fukouka,33.580412,130.396361,1,1,1,-9.0,0.0,1,0,Facility/Infrastructure Attack,Government (Diplomatic),Embassy/Consulate,U.S. Consulate,United States,Unknown,-99.0,-99.0,-99.0,Incendiary,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,1,Unknown,0.0,0.0,-9,-9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181686,201712310022,2017,12,31,0,Somalia,Sub-Saharan Africa,Middle Shebelle,Ceelka Geelow,2.359673,45.385034,1,1,0,1.0,0.0,1,0,Armed Assault,Military,Military Checkpoint,Checkpoint,Somalia,Al-Shabaab,-99.0,0.0,1.0,Firearms,1.0,0.0,0.0,2.0,0.0,0.0,-9,Unknown,0.0,-10.0,0,0
181687,201712310029,2017,12,31,0,Syria,Middle East & North Africa,Lattakia,Jableh,35.407278,35.942679,1,1,0,1.0,0.0,1,0,Bombing/Explosion,Military,Military Barracks/Base/Headquarters/Checkpost,Hmeymim Air Base,Russia,Muslim extremists,-99.0,0.0,0.0,Explosives,2.0,0.0,0.0,7.0,0.0,0.0,1,Unknown,0.0,-10.0,-9,-9
181688,201712310030,2017,12,31,0,Philippines,Southeast Asia,Maguindanao,Kubentog,6.900742,124.437908,1,1,1,0.0,0.0,1,0,Facility/Infrastructure Attack,Private Citizens & Property,House/Apartment/Residence,Houses,Philippines,Bangsamoro Islamic Freedom Movement (BIFM),-99.0,0.0,0.0,Incendiary,0.0,0.0,0.0,0.0,0.0,0.0,1,Unknown,0.0,-10.0,0,0
181689,201712310031,2017,12,31,0,India,South Asia,Manipur,Imphal,24.798346,93.940430,1,1,1,0.0,0.0,0,0,Bombing/Explosion,Government (General),Government Building/Facility/Office,Office,India,Unknown,-99.0,0.0,0.0,Explosives,0.0,0.0,0.0,0.0,0.0,0.0,-9,Unknown,0.0,-10.0,-9,-9


As a result of above operations I obtained a dataset without any missing values (I replaced them with appropriate values based on column's content).

### Exporting cleaned data into my Google Drive as *GTD_data_clean.csv*

In [21]:
#GTD_data.to_csv(r'/content/drive/MyDrive/data_warehouse_project/GTD_data_clean.csv')

In [22]:
#GTD_data.to_excel (r'/content/drive/MyDrive/data_warehouse_project/GTD_data_clean_excel.xlsx', index = None, header=True)

### Manual cleaning data in Google Spreadsheet 

After more careful analyis I decided to perform another manual cleaning on the data file. The cleaning process was about:

* dimensionality reduction in "country" column (f.e. West Germany and Eeast Germany -> Germany) 
* dimensionality reduction in "city" column (f.e. Rome district -> Rome)
* changing colun names for more convinient form (f.e. iyear -> year)






In [23]:
#GTD_data_clean = pd.read_csv("/content/drive/MyDrive/data_warehouse_project/GTD_data_clean_final.csv", encoding ='ISO-8859-1', low_memory=False)

In [26]:
GTD_data_clean.head()

,Unnamed: 0,eventid,year,month,day,extended,country,region,provstate,city,latitude,longitude,category1,category2,category3,doubtterr,multiple,success,suicide,attacktype,targtype,targsubtype,target_name,target_natlty,gname,nperps,nperpcap,claimed,weaptype1_txt,nkill,nkillus,nkillter,nwound,nwoundus,nwoundte,is_property_damage,property_damage,ishostkid,ransom,INT_LOG,INT_IDEO
0,0,197000000001,1970,7,2,0,Dominican Republic,Central America & Caribbean,Unknown,Santo Domingo,18.456792,-69.951164,1,1,1,0.0,0.0,1,0,Assassination,Private Citizens & Property,Named Civilian,Julio Guzman,Dominican Republic,MANO-D,-99.0,-99.0,-99.0,Unknown,1.0,-99.0,-99.0,0.0,-99.0,-99.0,0,Unknown,0.0,0.0,0,0
1,1,197000000002,1970,0,0,0,Mexico,North America,Federal,Mexico city,19.371887,-99.086624,1,1,1,0.0,0.0,1,0,Hostage Taking (Kidnapping),Government,"Diplomatic Personnel (outside of embassy, cons...","Nadine Chaval, daughter",Belgium,23rd of September Communist League,7.0,-99.0,-99.0,Unknown,0.0,-99.0,-99.0,0.0,-99.0,-99.0,0,Unknown,1.0,1.0,0,1
2,2,197001000001,1970,1,0,0,Philippines,Southeast Asia,Tarlac,Unknown,15.478598,120.599741,1,1,1,0.0,0.0,1,0,Assassination,Journalists & Media,Radio Journalist/Staff/Facility,Employee,United States,Unknown,-99.0,-99.0,-99.0,Unknown,1.0,-99.0,-99.0,0.0,-99.0,-99.0,0,Unknown,0.0,0.0,-9,-9
3,3,197001000002,1970,1,0,0,Greece,Western Europe,Attica,Athens,37.997490,23.762728,1,1,1,0.0,0.0,1,0,Bombing/Explosion,Government,Embassy/Consulate,U.S. Embassy,United States,Unknown,-99.0,-99.0,-99.0,Explosives,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,1,Unknown,0.0,0.0,-9,-9
4,4,197001000003,1970,1,0,0,Japan,East Asia,Fukouka,Fukouka,33.580412,130.396361,1,1,1,-9.0,0.0,1,0,Facility/Infrastructure Attack,Government,Embassy/Consulate,U.S. Consulate,United States,Unknown,-99.0,-99.0,-99.0,Incendiary,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,1,Unknown,0.0,0.0,-9,-9


In [31]:
GTD_data_clean.drop('Unnamed: 0', axis=1, inplace=True)

In [32]:
GTD_data_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181691 entries, 0 to 181690
Data columns (total 40 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   eventid             181691 non-null  int64  
 1   year                181691 non-null  int64  
 2   month               181691 non-null  int64  
 3   day                 181691 non-null  int64  
 4   extended            181691 non-null  int64  
 5   country             181691 non-null  object 
 6   region              181691 non-null  object 
 7   provstate           181691 non-null  object 
 8   city                181691 non-null  object 
 9   latitude            181691 non-null  float64
 10  longitude           181691 non-null  float64
 11  category1           181691 non-null  int64  
 12  category2           181691 non-null  int64  
 13  category3           181691 non-null  int64  
 14  doubtterr           181690 non-null  float64
 15  multiple            181690 non-nul

In [33]:
GTD_data_clean

,eventid,year,month,day,extended,country,region,provstate,city,latitude,longitude,category1,category2,category3,doubtterr,multiple,success,suicide,attacktype,targtype,targsubtype,target_name,target_natlty,gname,nperps,nperpcap,claimed,weaptype1_txt,nkill,nkillus,nkillter,nwound,nwoundus,nwoundte,is_property_damage,property_damage,ishostkid,ransom,INT_LOG,INT_IDEO
0,197000000001,1970,7,2,0,Dominican Republic,Central America & Caribbean,Unknown,Santo Domingo,18.456792,-69.951164,1,1,1,0.0,0.0,1,0,Assassination,Private Citizens & Property,Named Civilian,Julio Guzman,Dominican Republic,MANO-D,-99.0,-99.0,-99.0,Unknown,1.0,-99.0,-99.0,0.0,-99.0,-99.0,0,Unknown,0.0,0.0,0,0
1,197000000002,1970,0,0,0,Mexico,North America,Federal,Mexico city,19.371887,-99.086624,1,1,1,0.0,0.0,1,0,Hostage Taking (Kidnapping),Government,"Diplomatic Personnel (outside of embassy, cons...","Nadine Chaval, daughter",Belgium,23rd of September Communist League,7.0,-99.0,-99.0,Unknown,0.0,-99.0,-99.0,0.0,-99.0,-99.0,0,Unknown,1.0,1.0,0,1
2,197001000001,1970,1,0,0,Philippines,Southeast Asia,Tarlac,Unknown,15.478598,120.599741,1,1,1,0.0,0.0,1,0,Assassination,Journalists & Media,Radio Journalist/Staff/Facility,Employee,United States,Unknown,-99.0,-99.0,-99.0,Unknown,1.0,-99.0,-99.0,0.0,-99.0,-99.0,0,Unknown,0.0,0.0,-9,-9
3,197001000002,1970,1,0,0,Greece,Western Europe,Attica,Athens,37.997490,23.762728,1,1,1,0.0,0.0,1,0,Bombing/Explosion,Government,Embassy/Consulate,U.S. Embassy,United States,Unknown,-99.0,-99.0,-99.0,Explosives,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,1,Unknown,0.0,0.0,-9,-9
4,197001000003,1970,1,0,0,Japan,East Asia,Fukouka,Fukouka,33.580412,130.396361,1,1,1,-9.0,0.0,1,0,Facility/Infrastructure Attack,Government,Embassy/Consulate,U.S. Consulate,United States,Unknown,-99.0,-99.0,-99.0,Incendiary,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,1,Unknown,0.0,0.0,-9,-9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181686,201712310022,2017,12,31,0,Somalia,Sub-Saharan Africa,Middle Shebelle,Ceelka Geelow,2.359673,45.385034,1,1,0,1.0,0.0,1,0,Armed Assault,Military,Military Checkpoint,Checkpoint,Somalia,Al-Shabaab,-99.0,0.0,1.0,Firearms,1.0,0.0,0.0,2.0,0.0,0.0,-9,Unknown,0.0,-10.0,0,0
181687,201712310029,2017,12,31,0,Syria,Middle East & North Africa,Lattakia,Jableh,35.407278,35.942679,1,1,0,1.0,0.0,1,0,Bombing/Explosion,Military,Military Barracks/Base/Headquarters/Checkpost,Hmeymim Air Base,Russia,Muslim extremists,-99.0,0.0,0.0,Explosives,2.0,0.0,0.0,7.0,0.0,0.0,1,Unknown,0.0,-10.0,-9,-9
181688,201712310030,2017,12,31,0,Philippines,Southeast Asia,Maguindanao,Kubentog,6.900742,124.437908,1,1,1,0.0,0.0,1,0,Facility/Infrastructure Attack,Private Citizens & Property,House/Apartment/Residence,Houses,Philippines,Bangsamoro Islamic Freedom Movement (BIFM),-99.0,0.0,0.0,Incendiary,0.0,0.0,0.0,0.0,0.0,0.0,1,Unknown,0.0,-10.0,0,0
181689,201712310031,2017,12,31,0,India,South Asia,Manipur,Imphal,24.798346,93.940430,1,1,1,0.0,0.0,0,0,Bombing/Explosion,Government,Government Building/Facility/Office,Office,India,Unknown,-99.0,0.0,0.0,Explosives,0.0,0.0,0.0,0.0,0.0,0.0,-9,Unknown,0.0,-10.0,-9,-9


In [34]:
#GTD_data_clean.to_csv(r'/content/drive/MyDrive/data_warehouse_project/GTD_data_clean_final_ver.csv')